Version 1.0.0

# Introduction

In this programming assignment we will illustrate a very severe data leakage, that can often be found in competitions, where the pairs of object should be scored, e.g. predict $1$ if two objects belong to the same class and $0$ otherwise. 

The data in this assignment is taken from a real competition, and the funniest thing is that *we will not use training set at all* and achieve almost 100% accuracy score! We will just exploit the leakage.

Now go through the notebook and complete the assignment.

In [1]:
import numpy as np
import pandas as pd 
import scipy.sparse

# Load the data

Let's load the test data. Note, that we don't have any training data here, just test data. Moreover, *we will not even use any features* of test objects. All we need to solve this task is the file with the indices for the pairs, that we need to compare.

Let's load the data with test indices.

In [2]:
test = pd.read_csv('../readonly/data_leakages_data/test_pairs.csv')
test.head(10)

,pairId,FirstId,SecondId
0,0,1427,8053
1,1,17044,7681
2,2,19237,20966
3,3,8005,20765
4,4,16837,599
5,5,3657,12504
6,6,2836,7582
7,7,6136,6111
8,8,23295,9817
9,9,6621,7672


For example, we can think that there is a test dataset of images, and each image is assigned a unique `Id` from $0$ to $N-1$ (N -- is the number of images). In the dataframe from above `FirstId` and `SecondId` point to these `Id`'s and define pairs, that we should compare: e.g. do both images in the pair belong to the same class or not. So, for example for the first row: if images with `Id=1427` and `Id=8053` belong to the same class, we should predict $1$, and $0$ otherwise. 

But in our case we don't really care about the images, and how exactly we compare the images (as long as comparator is binary).  

**We suggest you to try to solve the puzzle yourself first.** You need to submit a `.csv` file with columns `pairId` and `Prediction` to the grader. The number of submissions allowed is made pretty huge to let you explore the data without worries. The returned score should be very close to $1$.

**If you do not want to think much** -- scroll down and follow the instructions below.

In [4]:
test.shape

(368550, 3)

In [5]:
len(test.FirstId.unique())

26325

In [6]:

len(test.SecondId.unique())

26310

In [7]:
test.FirstId.value_counts()

0        21
1261     21
1517     21
1658     21
123      21
379      21
635      21
891      21
1147     21
1146     21
1403     21
1659     21
124      21
380      21
636      21
892      21
1402     21
890      21
1148     21
377      21
632      21
888      21
1144     21
1400     21
1656     21
121      21
633      21
634      21
889      21
1145     21
         ..
25194     7
25875     7
25801     7
25079     7
25485     7
24938     7
25545     7
25602     7
24632     7
25934     7
25229     7
25619     7
25289     7
25327     7
24493     7
24295     7
25858     7
24973     7
25033     7
24574     7
24888     7
25678     7
24777     7
24527     7
25363     7
25579     7
25706     7
25003     7
24271     7
24899     7
Name: FirstId, Length: 26325, dtype: int64

In [8]:
test.SecondId.value_counts()

12194    23
12223    22
12279    22
12162    22
12141    22
12254    22
12137    22
12218    22
12181    22
12173    22
12214    22
12144    22
12166    22
12206    22
12121    22
12220    22
12177    22
12227    22
12133    22
12185    22
12280    22
12151    22
12186    22
12243    22
12175    22
12153    22
12114    22
12119    22
12213    22
12256    22
         ..
26142     2
26137     2
26139     2
26131     2
26143     2
26133     2
26134     2
26132     2
26136     2
26140     2
26135     2
26130     2
26144     2
26141     2
26138     2
26128     1
26115     1
26119     1
26124     1
26116     1
26129     1
26120     1
26127     1
26123     1
26117     1
26121     1
26126     1
26118     1
26125     1
26122     1
Name: SecondId, Length: 26310, dtype: int64

In [9]:
test.groupby('FirstId')['SecondId'].max()

FirstId
0        25963
1        24981
2        24982
3        24983
4        24984
5        24985
6        24986
7        24987
8        24988
9        24989
10       24990
11       25964
12       24992
13       24993
14       24994
15       24995
16       24996
17       24997
18       24998
19       24999
20       25000
21       25965
22       25002
23       25003
24       25004
25       25005
26       25006
27       25007
28       25008
29       25009
         ...  
26295    26312
26296    26323
26297    26324
26298    26315
26299    26316
26300    26317
26301    26318
26302    26319
26303    26320
26304    26321
26305    26322
26306    26308
26307    26309
26308    12257
26309    12168
26310    12078
26311     9391
26312     3012
26313    26323
26314    26324
26315     9237
26316    12279
26317    11844
26318     3045
26319    10462
26320     5683
26321     5697
26322    11297
26323    12000
26324     9353
Name: SecondId, Length: 26325, dtype: int64

# EDA and leakage intuition

As we already know, the key to discover data leakages is careful EDA. So let's start our work with some basic data exploration and build an intuition about the leakage.

First, check, how many different `id`s are there: concatenate `FirstId` and `SecondId` and print the number of unique elements. Also print minimum and maximum value for that vector.

In [10]:
# YOUR CODE GOES HERE
firstid = test.FirstId.unique()
secondid = test.SecondId.unique()
print('unique firstid :{}, max firstid:{}, min firstid:{}'.format(len(firstid), max(firstid),min(secondid)))
print('unique secondid :{}, max firstid:{}, min firstid:{}'.format(len(secondid),max(secondid),min(secondid)))

unique firstid :26325, max firstid:26324, min firstid:0
unique secondid :26310, max firstid:26324, min firstid:0


In [11]:
# YOUR CODE GOES HERE
inter_sec = np.intersect1d(firstid,secondid)
print('how many inter_sec id :{}'.format(len(inter_sec)))

how many inter_sec id :26310


and then print how many pairs we need to classify (it is basically the number of rows in the test set)

In [12]:

# YOUR CODE GOES HERE
distinct_rows = test[['FirstId','SecondId']].drop_duplicates()
print('numbers of row we need to classify :{}'.format(distinct_rows.shape[0]))

numbers of row we need to classify :368538


In [13]:
test[test[['FirstId','SecondId']].duplicated(keep = False)].sort_values(by='FirstId') # duplicated rows

,pairId,FirstId,SecondId
353199,353199,256,2551
105766,105766,256,2551
256914,256914,332,10404
124151,124151,332,10404
139487,139487,504,2799
101511,101511,504,2799
238033,238033,1420,8046
226316,226316,1420,8046
226778,226778,1980,8046
135741,135741,1980,8046


Now print, how many distinct pairs it would be possible to create out of all "images" in the dataset?   

In [14]:

# YOUR CODE GOES HERE
print('distinct pairs it would be possible out of all datasets:{}'.format(len(firstid)*len(secondid)-1))
# print('fraction:{:.4f} %'.format(100*distinct_rows.shape[0]/(len(firstid)*len(secondid))))

distinct pairs it would be possible out of all datasets:692610749


So the number of pairs we are given to classify is very very small compared to the total number of pairs. 

To exploit the leak we need to **assume (or prove)**, that the total number of positive pairs is small, compared to the total number of pairs. For example: think about an image dataset with $1000$ classes, $N$ images per class. Then if the task was to tell whether a pair of images belongs to the same class or not, we would have $1000\frac{N(N-1)}{2}$ positive pairs, while total number of pairs was $\frac{1000N(1000N - 1)}{2}$.

Another example: in [Quora competitition](https://www.kaggle.com/c/quora-question-pairs) the task was to classify whether a pair of qustions are duplicates of each other or not. Of course, total number of question pairs is very huge, while number of duplicates (positive pairs) is much much smaller.

Finally, let's get a fraction of pairs of class `1`. We just need to submit a constant prediction "all ones" and check the returned accuracy. Create a dataframe with columns `pairId` and `Prediction`, fill it and export it to `.csv` file. Then submit to grader and examine grader's output. 

In [15]:
# YOUR CODE GOES HERE
from collections import OrderedDict
pred = np.ones(len(test))

submission = pd.DataFrame(OrderedDict({'pairId': test.pairId,
                                      'Prediction': pred}))

submission.to_csv('submission.csv', index=False)

So, we assumed the total number of pairs is much higher than the number of positive pairs, but it is not the case for the test set. It means that the test set is constructed not by sampling random pairs, but with a specific sampling algorithm. Pairs of class `1` are oversampled.

Now think, how we can exploit this fact? What is the leak here? If you get it now, you may try to get to the final answer yourself, othewise you can follow the instructions below.   

In [16]:
from IPython.display import display

display(submission.head())
print()
!head 'submission.csv'

,pairId,Prediction
0,0,1.0
1,1,1.0
2,2,1.0
3,3,1.0
4,4,1.0



pairId,Prediction
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0


# Building a magic feature

In this section we will build a magic feature, that will solve the problem almost perfectly. The instructions will lead you to the correct solution, but please, try to explain the purpose of the steps we do to yourself -- it is very important.

## Incidence matrix

First, we need to build an [incidence matrix](https://en.wikipedia.org/wiki/Incidence_matrix). You can think of pairs `(FirstId, SecondId)` as of edges in an undirected graph. 

The incidence matrix is a matrix of size `(maxId + 1, maxId + 1)`, where each row (column) `i` corresponds `i-th` `Id`. In this matrix we put the value `1` to the position `[i, j]`, if and only if a pair `(i, j)` or `(j, i)` is present in  a given set of pais `(FirstId, SecondId)`. All the other elements in the incidence matrix are zeros.   

**Important!** The incidence matrices are typically very very sparse (small number of non-zero values). At the same time incidence matrices are usually huge in terms of total number of elements, and it is **impossible to store them in memory in dense format**. But due to their sparsity incidence matrices **can be easily represented as sparse matrices**. If you are not familiar with sparse matrices, please see [wiki](https://en.wikipedia.org/wiki/Sparse_matrix) and [scipy.sparse reference](https://docs.scipy.org/doc/scipy/reference/sparse.html). Please, use any of `scipy.sparse` constructors to build incidence matrix. 

For example, you can use this constructor: `scipy.sparse.coo_matrix((data, (i, j)))`. We highly recommend to learn to use different `scipy.sparse` constuctors, and matrices types, but if you feel you don't want to use them, you can always build this matrix with a simple `for` loop. You will need first to create a matrix using `scipy.sparse.coo_matrix((M, N), [dtype])` with an appropriate shape `(M, N)` and then iterate through `(FirstId, SecondId)` pairs and fill corresponding elements in matrix with ones. 

**Note**, that the matrix should be symmetric and consist only of zeros and ones. It is a way to check yourself.

In [17]:

indices = np.unique( \
                   np.concatenate( \
                                  [test[['FirstId', 'SecondId']].values,
                                   test[['SecondId', 'FirstId']].values
                                  ]
                                 ), axis=0
                   )

inc_mat = scipy.sparse.coo_matrix( \
                                  (np.ones(len(indices)), 
                                  (indices[:, 0], indices[:, 1]))
                                 )# YOUR CODE GOES HERE (but probably you will need to write few more lines before)

# Sanity checks
assert inc_mat.max() == 1
assert inc_mat.sum() == 736872

In [18]:
print(indices.shape)
print(indices[:5, 0])
print(indices[:5, 1])
print()
print(type(inc_mat))
print(inc_mat.shape)

(736872, 2)
[0 0 0 0 0]
[16131 14856  6157   552  6453]

<class 'scipy.sparse.coo.coo_matrix'>
(26325, 26325)


It is convenient to have matrix in `csr` format eventually.

In [19]:
inc_mat = inc_mat.tocsr()

## Now build the magic feature

Why did we build the incidence matrix? We can think of the rows in this matix as of representations for the objects. `i-th` row is a representation for an object with `Id = i`. Then, to measure similarity between two objects we can measure similarity between their representations. And we will see, that such representations are very good.

Now select the rows from the incidence matrix, that correspond to `test.FirstId`'s, and `test.SecondId`'s.

In [21]:

# Note, scipy goes crazy if a matrix is indexed with pandas' series. 
# So do not forget to convert `pd.series` to `np.array`
# These lines should normally run very quickly 

rows_FirstId   = inc_mat[test['FirstId'].values]# YOUR CODE GOES HERE
rows_SecondId  = inc_mat[test['SecondId'].values]# YOUR CODE GOES HERE

In [22]:
print(type(rows_FirstId))
print(rows_FirstId.shape)
print(rows_SecondId.shape)

<class 'scipy.sparse.csr.csr_matrix'>
(368550, 26325)
(368550, 26325)


Our magic feature will be the *dot product* between representations of a pair of objects. Dot product can be regarded as similarity measure -- for our non-negative representations the dot product is close to 0 when the representations are different, and is huge, when representations are similar. 

Now compute dot product between corresponding rows in `rows_FirstId` and `rows_SecondId` matrices.

In [26]:
# Note, that in order to do pointwise multiplication in scipy.sparse you need to use function `multiply`
# regular `*` corresponds to matrix-matrix multiplication

f = np.array(rows_FirstId.multiply(rows_SecondId).sum(axis=1)).squeeze()# YOUR CODE GOES HERE

# Sanity check
assert f.shape == (368550, )

That is it! **We've built our magic feature.** 

# From magic feature to binary predictions

But how do we convert this feature into binary predictions? We do not have a train set to learn a model, but we have a piece of information about test set: the baseline accuracy score that you got, when submitting constant. And we also have a very strong considerations about the data generative process, so probably we will be fine even without a training set. 

We may try to choose a thresold, and set the predictions to 1, if the feature value `f` is higer than the threshold, and 0 otherwise. What threshold would you choose? 

How do we find a right threshold? Let's first examine this feature: print frequencies (or counts) of each value in the feature `f`.

In [28]:

# For example use `np.unique` function, check for flags

print(np.unique(f, return_counts=True)) # YOUR CODE GOES HERE

(array([ 14.,  15.,  19.,  20.,  21.,  28.,  35.]), array([183279,    852,    546, 183799,      6,     54,     14]))


Do you see how this feature clusters the pairs? Maybe you can guess a good threshold by looking at the values? 

In fact, in other situations it can be not that obvious, but in general to pick a threshold you only need to remember the score of your baseline submission and use this information. Do you understand why and how?  

Choose a threshold below: 

In [29]:
pred = f > 19 # SET THRESHOLD HERE

# Finally, let's create a submission

In [30]:

submission = test.loc[:,['pairId']]
submission['Prediction'] = pred.astype(int)

submission.to_csv('submission.csv', index=False)

Now submit it to the grader! It is not possible to submit directly from this notebook, as we need to submit a `csv` file, not a single number (limitation of Coursera platform). 

To download `submission.csv` file that you've just produced <a href='./submission.csv'>click here</a> (if the link opens in browser, right-click on it and shoose "Save link as"). Then go to [assignment page](https://www.coursera.org/learn/competitive-data-science/programming/KsASv/data-leakages/submission) and submit your `.csv` file in 'My submission' tab.


If you did everything right, the score should be very high.

**Finally:** try to explain to yourself, why the whole thing worked out. In fact, there is no magic in this feature, and the idea to use rows in the incidence matrix can be intuitively justified.

# Bonus

Interestingly, it is not the only leak in this dataset. There is another totally different way to get almost 100% accuracy. Try to find it!

## Leak exploited

The objective of this challenge is to predict 1 if two images correspond to the same class and 0 otherwise. The competition gives 368550 couples of images in order to predict their result. Since there are 26,325 different images ids the total number of possible combinations would be (26,325(26,325-1))/2. This means the amount of couples of images given is too small and the way they are organized is not random.

To take advantage of this data leak we create an incidence matrix which could be seen also as a vectorial representation for each image, the similarity between those vectors calculated as the dot product between vectors shows that the test images used during the test data set is oversampled and overlaps with more frequency when the images are from the same class, allowing to get almost a perfect score from this feature.